In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import os
import json
from pandas.io.json import json_normalize

pd.set_option('display.max_columns', None)

In [2]:
# Codes to read data as well as convert json columns
def load_data(csv_path='./train_v2.csv', n_rows=None):
    # Read column names from file
    cols = list(pd.read_csv(csv_path, nrows=1))
    # Columns to be skipped when reading
    skip_cols = ['customDimensions', 'hits', 'socialEngagementType']
    # json columns
    json_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']

    # Importing the dataset
    df = pd.read_csv(csv_path,
                     # Columns to be used
                     usecols=[col for col in cols if col not in skip_cols],
                     # Loading the json columns properly
                     converters={column: json.loads for column in json_cols},
                     # Transforming this column to string
                     dtype={'fullVisitorId': 'str'},
                     nrows=n_rows)  # Number of rows that will be imported randomly

    # Loop to convert json columns to tables
    for column in json_cols:
        # This will normalize and set the json to a table
        column_as_df = json_normalize(df[column])
        # The column name is set using the category and subcategory of json columns
        column_as_df.columns = [
            f'{column}_{subcolumn}' for subcolumn in column_as_df.columns]
        # After extracting the values, we drop the original columns
        df = df.drop(column, axis=1).merge(
            column_as_df, right_index=True, left_index=True)

    # Printing the shape of dataframes that was imported
    print(f'Loaded {os.path.basename(csv_path)}. Shape: {df.shape}')
    return df  # Returning the df after importing and transforming

In [3]:
%%time

# Load the data
df_train = load_data()

Loaded train_v2.csv. Shape: (1708337, 57)
CPU times: user 6min 27s, sys: 5min 27s, total: 11min 54s
Wall time: 17min 15s


In [4]:
df_train.head()  # First five rows

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device_browser,device_browserSize,device_browserVersion,device_deviceCategory,device_flashVersion,device_isMobile,device_language,device_mobileDeviceBranding,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_mobileDeviceModel,device_mobileInputSelector,device_operatingSystem,device_operatingSystemVersion,device_screenColors,device_screenResolution,geoNetwork_city,geoNetwork_cityId,geoNetwork_continent,geoNetwork_country,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_networkLocation,geoNetwork_region,geoNetwork_subContinent,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,totals_visits,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_campaignCode,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20171016,3162355547410993243,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,Western Europe,1,1,1,1,1,NaN,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,water bottle,organic,NaN,google
1,Referral,20171016,8934116514970143966,1508176307,6,1508176307,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Chrome OS,not available in demo dataset,not available in demo dataset,not available in demo dataset,Cupertino,not available in demo dataset,Americas,United States,not available in demo dataset,not available in demo dataset,San Francisco-Oakland-San Jose CA,(not set),not available in demo dataset,California,Northern America,NaN,2,NaN,2,2,28,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com
2,Direct,20171016,7992466427990357681,1508201613,1,1508201613,Chrome,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Americas,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,windjammercable.net,not available in demo dataset,not available in demo dataset,Northern America,NaN,2,1,2,1,38,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,True,NaN,(none),NaN,(direct)
3,Organic Search,20171016,9075655783635761930,1508169851,1,1508169851,Chrome,not available in demo dataset,not available in demo dataset,desktop,not availabl

In [5]:
# Function to find columns with a single value (non-informative columns) and drop them
def uniq_val_col_drop(df):
    uniq_value_cols = [
        col for col in df.columns if df[col].nunique(dropna=False) == 1]
    df.drop(uniq_value_cols, axis=1, inplace=True)
    print(f'{len(uniq_value_cols)} column[s] dropped!')
    return df

In [6]:
# Dropping columns with a unique value from the data
df_train = uniq_val_col_drop(df_train)

df_train.head()  # First five rows

18 column[s] dropped!


,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,geoNetwork_city,geoNetwork_continent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_region,geoNetwork_subContinent,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_campaignCode,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20171016,3162355547410993243,1508198450,1,1508198450,Firefox,desktop,False,Windows,not available in demo dataset,Europe,Germany,not available in demo dataset,(not set),not available in demo dataset,Western Europe,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,water bottle,organic,NaN,google
1,Referral,20171016,8934116514970143966,1508176307,6,1508176307,Chrome,desktop,False,Chrome OS,Cupertino,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,NaN,2,NaN,2,2,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com
2,Direct,20171016,7992466427990357681,1508201613,1,1508201613,Chrome,mobile,True,Android,not available in demo dataset,Americas,United States,not available in demo dataset,windjammercable.net,not available in demo dataset,Northern America,NaN,2,1,2,1,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,True,NaN,(none),NaN,(direct)
3,Organic Search,20171016,9075655783635761930,1508169851,1,1508169851,Chrome,desktop,False,Windows,not available in demo dataset,Asia,Turkey,not available in demo dataset,unknown.unknown,not available in demo dataset,Western Asia,NaN,2,1,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
4,Organic Search,20171016,6960673291025684308,1508190552,1,1508190552,Chrome,desktop,False,Windows,not available in demo dataset,Americas,Mexico,not available in demo dataset,prod-infinitum.com.mx,not available in demo dataset,Central America,NaN,2,1,2,1,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google


In [7]:
# Determining the number of missing values
def missing_values(df):
    count_missing = df.isnull().sum().sort_values(
        ascending=False)  # Counting missing values and sorting
    # Percentage of rows with a missing value in each column
    percent_missing = count_missing / len(df) * 100
    null = pd.concat([count_missing, percent_missing], axis=1, keys=[
                     'count_missing', 'percent_missing'])  # Concatenating count and percent

    percent_nonnull_tranrev = 100 - percent_missing['totals_transactionRevenue']

    print("Columns with at least one missing value: ")
    # Returning info of columns with at least one value
    print(null[(null['count_missing'] != 0)])
    print(f'\n Percentage of sessions with transactions data: {percent_nonnull_tranrev:.2f}%') # Percentage of sessions with transactions
    return

In [8]:
missing_values(df_train)  # Info on missing value in the data

Columns with at least one missing value: 
                                              count_missing  percent_missing
trafficSource_campaignCode                          1708336        99.999941
totals_transactionRevenue                           1689823        98.916256
totals_totalTransactionRevenue                      1689823        98.916256
totals_transactions                                 1689778        98.913622
trafficSource_adContent                             1643600        96.210525
trafficSource_adwordsClickInfo.slot                 1633063        95.593727
trafficSource_adwordsClickInfo.page                 1633063        95.593727
trafficSource_adwordsClickInfo.isVideoAd            1633063        95.593727
trafficSource_adwordsClickInfo.adNetworkType        1633063        95.593727
trafficSource_adwordsClickInfo.gclId                1632914        95.585005
trafficSource_isTrueDirect                          1173819        68.711209
trafficSource_referralPath        

In [9]:
# Dealing with missing data and changing column types
def fill_na(df):    # fillna numeric feature
    df['totals_pageviews'].fillna(1, inplace=True)  # filling NA's with 1
    df['totals_newVisits'].fillna(0, inplace=True)  # filling NA's with 0
    df['totals_bounces'].fillna(0, inplace=True)  # filling NA's with 0
    df['trafficSource_isTrueDirect'].fillna(
        False, inplace=True)  # filling boolean with False
    df['trafficSource_adwordsClickInfo.isVideoAd'].fillna(
        True, inplace=True)  # filling boolean with True
    df['totals_transactions'] = df['totals_transactions'].fillna(
        0.).astype(int)  # filling NA with 0
    df['totals_transactionRevenue'] = df['totals_transactionRevenue'].fillna(
        0.).astype(float)  # filling NA with 0
    df['totals_totalTransactionRevenue'] = df['totals_totalTransactionRevenue'].fillna(
        0.).astype(float)  # filling NA with 0
    df['totals_pageviews'] = df['totals_pageviews'].astype(
        int)  # setting numerical column as integer
    df['totals_newVisits'] = df['totals_newVisits'].astype(
        int)  # setting numerical column as integer
    df['totals_bounces'] = df['totals_bounces'].astype(
        int)  # setting numerical column as integer
    df['totals_hits'] = df["totals_hits"].astype(
        int)  # setting numerical to float
    df['totals_newVisits'] = df['totals_newVisits'].astype(
        int)  # seting as int
    return df  # Return the transformed dataframe

In [10]:
# Dealing with missing data
df_train = fill_na(df_train)

In [11]:
df_train.dtypes

channelGrouping                                  object
date                                              int64
fullVisitorId                                    object
visitId                                           int64
visitNumber                                       int64
visitStartTime                                    int64
device_browser                                   object
device_deviceCategory                            object
device_isMobile                                    bool
device_operatingSystem                           object
geoNetwork_city                                  object
geoNetwork_continent                             object
geoNetwork_country                               object
geoNetwork_metro                                 object
geoNetwork_networkDomain                         object
geoNetwork_region                                object
geoNetwork_subContinent                          object
totals_bounces                                  

In [12]:
# Function to find columns with a single value (non-informative columns) and drop them
def uniq_col_drop_final(df):
    uniq_value_cols = [col for col in df.columns if df[col].nunique() == 1]
    df.drop(uniq_value_cols, axis=1, inplace=True)
    print(f'{len(uniq_value_cols)} column[s] dropped!')
    return df

In [13]:
# Dropping columns with a unique value from the data
df_train = uniq_col_drop_final(df_train)

df_train.head()  # First three rows

1 column[s] dropped!


,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,geoNetwork_city,geoNetwork_continent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_region,geoNetwork_subContinent,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20171016,3162355547410993243,1508198450,1,1508198450,Firefox,desktop,False,Windows,not available in demo dataset,Europe,Germany,not available in demo dataset,(not set),not available in demo dataset,Western Europe,1,1,1,1,1,NaN,0.0,0.0,0,NaN,NaN,NaN,True,NaN,NaN,(not set),False,water bottle,organic,NaN,google
1,Referral,20171016,8934116514970143966,1508176307,6,1508176307,Chrome,desktop,False,Chrome OS,Cupertino,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,0,2,0,2,2,28,0.0,0.0,0,NaN,NaN,NaN,True,NaN,NaN,(not set),False,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com
2,Direct,20171016,7992466427990357681,1508201613,1,1508201613,Chrome,mobile,True,Android,not available in demo dataset,Americas,United States,not available in demo dataset,windjammercable.net,not available in demo dataset,Northern America,0,2,1,2,1,38,0.0,0.0,0,NaN,NaN,NaN,True,NaN,NaN,(not set),True,NaN,(none),NaN,(direct)
3,Organic Search,20171016,9075655783635761930,1508169851,1,1508169851,Chrome,desktop,False,Windows,not available in demo dataset,Asia,Turkey,not available in demo dataset,unknown.unknown,not available in demo dataset,Western Asia,0,2,1,2,1,1,0.0,0.0,0,NaN,NaN,NaN,True,NaN,NaN,(not set),False,(not provided),organic,NaN,google
4,Organic Search,20171016,6960673291025684308,1508190552,1,1508190552,Chrome,desktop,False,Windows,not available in demo dataset,Americas,Mexico,not available in demo dataset,prod-infinitum.com.mx,not available in demo dataset,Central America,0,2,1,2,1,52,0.0,0.0,0,NaN,NaN,NaN,True,NaN,NaN,(not set),False,(not provided),organic,NaN,google


In [14]:
%%time

# Save the flattened data set
df_train.to_csv("./train-flattened.csv", index=False)

CPU times: user 39.7 s, sys: 6.15 s, total: 45.8 s
Wall time: 48 s


In [15]:
%%time

# Read the flattened data set to check it and also the time
train_flattened = pd.read_csv(
    './train-flattened.csv', dtype={'fullVisitorId': 'str'})

CPU times: user 11.6 s, sys: 2.06 s, total: 13.7 s
Wall time: 14.9 s


In [16]:
train_flattened.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,geoNetwork_city,geoNetwork_continent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_region,geoNetwork_subContinent,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20171016,3162355547410993243,1508198450,1,1508198450,Firefox,desktop,False,Windows,not available in demo dataset,Europe,Germany,not available in demo dataset,(not set),not available in demo dataset,Western Europe,1,1,1,1,1.0,NaN,0.0,0.0,0,NaN,NaN,NaN,True,NaN,NaN,(not set),False,water bottle,organic,NaN,google
1,Referral,20171016,8934116514970143966,1508176307,6,1508176307,Chrome,desktop,False,Chrome OS,Cupertino,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,0,2,0,2,2.0,28.0,0.0,0.0,0,NaN,NaN,NaN,True,NaN,NaN,(not set),False,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com
2,Direct,20171016,7992466427990357681,1508201613,1,1508201613,Chrome,mobile,True,Android,not available in demo dataset,Americas,United States,not available in demo dataset,windjammercable.net,not available in demo dataset,Northern America,0,2,1,2,1.0,38.0,0.0,0.0,0,NaN,NaN,NaN,True,NaN,NaN,(not set),True,NaN,(none),NaN,(direct)
3,Organic Search,20171016,9075655783635761930,1508169851,1,1508169851,Chrome,desktop,False,Windows,not available in demo dataset,Asia,Turkey,not available in demo dataset,unknown.unknown,not available in demo dataset,Western Asia,0,2,1,2,1.0,1.0,0.0,0.0,0,NaN,NaN,NaN,True,NaN,NaN,(not set),False,(not provided),organic,NaN,google
4,Organic Search,20171016,6960673291025684308,1508190552,1,1508190552,Chrome,desktop,False,Windows,not available in demo dataset,Americas,Mexico,not available in demo dataset,prod-infinitum.com.mx,not available in demo dataset,Central America,0,2,1,2,1.0,52.0,0.0,0.0,0,NaN,NaN,NaN,True,NaN,NaN,(not set),False,(not provided),organic,NaN,google
